In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests

In [2]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')
#response.text[:1500]

In [3]:
data = response.text.split('\n')
data = data[253:]
#len(data)
data = " ".join(data)

In [4]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [5]:
#len(tokens)
len(set(tokens))

27956

Let's use a set of 50 words to predict the 51st word

In [6]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000:
    break

print(len(lines))
#lines[0]

199951


In [14]:
type(lines)

list

Building the LSTM model

In [7]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [9]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [10]:
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

50

In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            650450    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 13009)             1313909   
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, batch_size = 256, epochs = 100)

Epoch 1/100
782/782 [==============================] - 59s 65ms/step - loss: 6.8741 - accuracy: 0.0309
Epoch 2/100
782/782 [==============================] - 49s 63ms/step - loss: 6.4981 - accuracy: 0.0445
Epoch 3/100
782/782 [==============================] - 48s 61ms/step - loss: 6.2776 - accuracy: 0.0623
Epoch 4/100
782/782 [==============================] - 46s 58ms/step - loss: 6.0934 - accuracy: 0.0751
Epoch 5/100
782/782 [==============================] - 46s 58ms/step - loss: 5.9539 - accuracy: 0.0849
Epoch 6/100
782/782 [==============================] - 46s 58ms/step - loss: 5.8356 - accuracy: 0.0931
Epoch 7/100
782/782 [==============================] - 46s 58ms/step - loss: 5.7259 - accuracy: 0.1004
Epoch 8/100
782/782 [==============================] - 46s 58ms/step - loss: 5.6289 - accuracy: 0.1055
Epoch 9/100
782/782 [==============================] - 45s 57ms/step - loss: 5.5398 - accuracy: 0.1089
Epoch 10/100
782/782 [==============================] - 45s 58ms/step - l

In [15]:
seed_text=lines[12343]
seed_text

'home of love if i have ranged like him that travels i return again just to the time not with the time exchanged so that my self bring water for my stain never believe though in my nature reigned all frailties that besiege all kinds of blood that it could so'

In [16]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [17]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'preposterously be stained to leave to fail and by the fatness of our senseless judgment and ourself and knewst the broken world the virginal infectious second servant of the world and let me hear of him and about the consuls worthiness of the rangd of them brats in a perpetual dulness let their hast been within and we in solemn hours qualifies the bravery of the kings that milks and does the prisond walls tables therein and lack extinct in take and feeding doth eat triumphant merit more strong horrible shaming a crooked sentence of barren senators and desprate creature'